<a href="https://colab.research.google.com/github/mobinhs121/deep_learning/blob/main/Gray2RGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D
import pandas as pd
import os
from sklearn.utils import shuffle

def build_model(input_shape):
    inputs = Input(shape=input_shape)

    # Custom VGG-16-like encoder for grayscale input
    encoder = tf.keras.Sequential([
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
    ])

    encoder_output = encoder(inputs)

    # Decoder
    decoder = Conv2D(256, (3, 3), activation='relu', padding='same')(encoder_output)
    decoder = UpSampling2D((2, 2))(decoder)
    decoder = Conv2D(128, (3, 3), activation='relu', padding='same')(decoder)
    decoder = UpSampling2D((2, 2))(decoder)
    decoder = Conv2D(64, (3, 3), activation='relu', padding='same')(decoder)
    decoder = UpSampling2D((2, 2))(decoder)
    decoder = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(decoder)

    model = tf.keras.Model(inputs, decoder)
    return model

input_shape = (224, 224, 1)  # Grayscale image input
model = build_model(input_shape)
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 1)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 28, 28, 256)       1734336   
                                                                 
 conv2d_18 (Conv2D)          (None, 28, 28, 256)       590080    
                                                                 
 up_sampling2d_3 (UpSampling  (None, 56, 56, 256)      0         
 2D)                                                             
                                                                 
 conv2d_19 (Conv2D)          (None, 56, 56, 128)       295040    
                                                                 
 up_sampling2d_4 (UpSampling  (None, 112, 112, 128)    0         
 2D)                                                       

In [2]:
!pip install kaggle

from google.colab import files
files.upload()  # Upload your Kaggle API key JSON file

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d benjaminbirchhansen/img-net-10percent
!unzip dataset_name.zip -d benjaminbirchhansen/img-net-10percent

Saving kaggle.json to kaggle.json
100% 4.14G/4.14G [02:11<00:00, 42.1MB/s]
100% 4.14G/4.14G [02:12<00:00, 33.7MB/s]
unzip:  cannot find or open dataset_name.zip, dataset_name.zip.zip or dataset_name.zip.ZIP.


In [3]:
!unzip img-net-10percent.zip

Streaming output truncated to the last 5000 lines.
  inflating: sampled_data_train/n03908714_3886.JPEG  
  inflating: sampled_data_train/n03908714_3978.JPEG  
  inflating: sampled_data_train/n03908714_4036.JPEG  
  inflating: sampled_data_train/n03908714_4321.JPEG  
  inflating: sampled_data_train/n03908714_4434.JPEG  
  inflating: sampled_data_train/n03908714_4471.JPEG  
  inflating: sampled_data_train/n03908714_4683.JPEG  
  inflating: sampled_data_train/n03908714_487.JPEG  
  inflating: sampled_data_train/n03908714_5175.JPEG  
  inflating: sampled_data_train/n03908714_530.JPEG  
  inflating: sampled_data_train/n03908714_5337.JPEG  
  inflating: sampled_data_train/n03908714_5425.JPEG  
  inflating: sampled_data_train/n03908714_5458.JPEG  
  inflating: sampled_data_train/n03908714_5548.JPEG  
  inflating: sampled_data_train/n03908714_5558.JPEG  
  inflating: sampled_data_train/n03908714_5646.JPEG  
  inflating: sampled_data_train/n03908714_6047.JPEG  
  inflating: sampled_data_train/n

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Create a dataframe with filenames
file_list = os.listdir('/content/sampled_data_train')
df = pd.DataFrame({'filename': file_list})

# Shuffle the dataframe
df = shuffle(df)
datagen = ImageDataGenerator(rescale=1.0 / 255)

batch_size = 32
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory='/content/sampled_data_train',  # Path to the directory containing images
    x_col='filename',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='input',  # Grayscale input images
    color_mode='grayscale'
)


Found 41317 validated image filenames.


In [10]:
# Define train and validation data generators
from sklearn.model_selection import train_test_split

# Assuming you have a dataframe named 'df' with 'filename' column
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='/content/sampled_data_train',  # Path to the directory containing train images
    x_col='filename',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='input',
    color_mode='grayscale'
)

val_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='/content/sampled_data_train',  # Path to the directory containing validation images
    x_col='filename',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='input',
    color_mode='grayscale'
)

# Compile and train the model
model.compile(optimizer='adam', loss='mse')

epochs = 20
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=epochs
)


Found 33053 validated image filenames.
Found 8264 validated image filenames.
Epoch 1/20
1032/1032 [==============================] - 523s 479ms/step - loss: 0.0092 - val_loss: 0.0056
Epoch 2/20
1032/1032 [==============================] - 480s 465ms/step - loss: 0.0051 - val_loss: 0.0045
Epoch 3/20
1032/1032 [==============================] - 478s 463ms/step - loss: 0.0045 - val_loss: 0.0043
Epoch 4/20
1032/1032 [==============================] - 477s 462ms/step - loss: 0.0041 - val_loss: 0.0038
Epoch 5/20
1032/1032 [==============================] - 477s 462ms/step - loss: 0.0037 - val_loss: 0.0038
Epoch 6/20
1032/1032 [==============================] - 481s 466ms/step - loss: 0.0034 - val_loss: 0.0032
Epoch 7/20
1032/1032 [==============================] - 481s 466ms/step - loss: 0.0032 - val_loss: 0.0031
Epoch 8/20
1032/1032 [==============================] - 478s 463ms/step - loss: 0.0030 - val_loss: 0.0030
Epoch 9/20
1032/1032 [==============================] - 478s 463ms/step - l

In [12]:
model.save('gray2rgb.keras')

In [14]:
!du -sh gray2rgb.keras

31M	gray2rgb.keras


In [16]:
!du -sh sampled_data_train

4.3G	sampled_data_train


In [17]:
from google.colab import drive

In [18]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [20]:
!git add gray2rgb.keras

In [21]:
!git commit -m 'model added'

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@a48c110595b5.(none)')


In [23]:
!git remote add origin https://github.com/mobinhs121/Gray2RGB

In [25]:
!git config --global user.email 'mobin.h310@gmail.com'